# Aleket Faster R-CNN training notebook

In [ ]:
%pip install pillow
%pip install numpy<2.0
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu124
%pip install matplotlib
%pip install gdown
%pip install tqdm

from IPython.display import clear_output

clear_output(wait=False)

print("ALL DEPENDENCIES INSTALLED")

In [1]:
# IMPORTS

# Standard Library
import random

# Third-Party Libraries
import numpy as np
from matplotlib import pyplot as plt

# Torch
import torch
from torchvision.models.detection import FasterRCNN, fasterrcnn_resnet50_fpn, fasterrcnn_mobilenet_v3_large_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Utils
from aleket_dataset import AleketDataset, download_dataset
from utils import split_dataset
from training_and_evaluation import train
from dataset_statisics import visualize_example_bboxes
from run_params import RunParams

In [2]:
# Device Selection
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Random Seed for Reproducibility
SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
np_generator = np.random.default_rng(SEED)

# Dataset split
DATASET_FRACTION = 0.1
VALIDATION_FRACTION = 0.2
dataset = AleketDataset(download_dataset("dataset_patched", ""), img_size=1024)
train_set, val_set = split_dataset(dataset, DATASET_FRACTION, VALIDATION_FRACTION, np_generator)

def augment_example(ds):
    examples = visualize_example_bboxes(ds, image_ids_to_visualize=range(4))
    fig=plt.figure(figsize=(15, 3))
    columns = 4
    rows = 1
    for i in range(1, columns*rows +1):
        fig.add_subplot(rows, columns, i)
        plt.imshow(examples[i-1])
    plt.show()
    
    
def get_model(device: torch.device) -> FasterRCNN:
    """
    Loads a pretrained Faster R-CNN ResNet-50 FPN model and modifies the classification head 
    to accommodate the specified number of classes in dataset (3 - including background).
    Args:
        device (torch.device): The device to move the model to (e.g., 'cuda' or 'cpu').

    Returns:
        FasterRCNN: The Faster R-CNN model with the modified classification head.
    """
    model = fasterrcnn_resnet50_fpn(
        weights = "DEFAULT",
        trainable_backbone_layers = 3,
        box_nms_thresh=0.7,
        box_detections_per_img=200
    )
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = (
        FastRCNNPredictor(
            in_features, 3
        )
    )
    return model.to(device)

Using device: cuda
Dataset loaded from dataset_patched


In [3]:

model = get_model(device)
params = RunParams(
    run_name="run1",
    batch_size=16,
    dataloader_workers=16, 
    total_epochs=1000,
    augmentation={
        "horizontal_flip": {
            "p": 0.5
        },
        "vertical_flip": {
            "p": 0.5
        },
        "perspective": {
            "distortion_scale": 0.2,
            "p": 0.5
        },
        "rotation": {
            "degrees": 15,
            "expand": True
        },
        "color_jitter": {
            "brightness": 0.2,
            "contrast": 0.1,
            "saturation": 0.05
        }
    },
    optimizer={
        "lr": 0.001,
        "momentum": 0.9,
        "weight_decay": 0.0001
    },
    lr_scheduler={
        "factor": 0.5,
        "patience": 10,
        "min_lr": 0.0001
    },
    validation_set=val_set,
    train_set=train_set
    )

train(model, dataset, params, device, checkpoints=True)



Epoch 1/1000; Learning rate: 0.001


Training batches:   0%|          | 0/33 [00:00<?, ?it/s]ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
Training batches:   0%|          | 0/33 [00:52<?, ?it/s]


KeyboardInterrupt: 